###**Q1**.  Use [Lucidchart](https://sjsu.edu/it/services/applications/lucidchart.php) to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [45]:
import pandas as pd
import sqlite3


In [46]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
# Your code here . . .
conn = sqlite3.connect("/content/store.db")
cursor = conn.cursor()
cursor.execute('''CREATE TABLE product (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price REAL
)
''')
cursor.execute('''CREATE TABLE customer (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT NOT NULL
)
''')
cursor.execute('''CREATE TABLE purchase_order (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    date TEXT NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
)
''')
cursor.execute('''CREATE TABLE order_item (
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY (order_id) REFERENCES purchase_order(order_id),
    FOREIGN KEY (product_id) REFERENCES product(product_id)
)
''')

In [47]:
# 2.3 Load the data in the csv files into the corresponding table.
# Your code here . . .
prod_data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/product.csv')
customer_data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv')
order_item_data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv')
purchase_data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv')

In [48]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
# Your code here . . .
prod_data.to_sql('product', conn, if_exists='append', index=False)
customer_data.to_sql('customer', conn, if_exists='append', index=False)
order_item_data.to_sql('order_item', conn, if_exists='append', index=False)
purchase_data.to_sql('purchase_order', conn, if_exists='append', index=False)

3

In [49]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .
rows = cursor.execute("SELECT * FROM product").fetchall()
print('data from product table')
for row in rows:
  print(row)


data = cursor.execute("SELECT * FROM customer").fetchall()
print('data from customer table')
for row in data:
  print(row)


data = cursor.execute(f"SELECT * FROM purchase_order").fetchall()
print('data from purchase_order table')
for row in data:
  print(row)

data = cursor.execute(f"SELECT * FROM order_item").fetchall()
print('data from order_items table')
for row in data:
  print(row)





data from product table
(0, 'bicycle', 400.0)
(1, 'helmet', 45.0)
(2, 'gloves', 23.0)
(3, 'chain', 48.0)
data from customer table
(0, 'Wendy Lee', 'wlee@bike.com')
(1, 'Jason Brown', 'jb@speed.com')
(2, 'Harry Potter', 'hp@hogwarts.edu')
(3, 'Godric Gryffindor', 'gg@hogwards.edu')
data from purchase_order table
(0, 0, '2020-10-19')
(1, 0, '2020-10-20')
(2, 1, '2020-10-20')
data from order_items table
(0, 0, 1)
(0, 1, 1)
(1, 2, 2)
(1, 3, 1)


In [50]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Query 1: ???
customer_id = 1
query = '''
SELECT purchase_order.date, product.name, order_item.quantity
FROM purchase_order
JOIN order_item ON purchase_order.order_id = order_item.order_id
JOIN product ON order_item.product_id = product.product_id
WHERE purchase_order.customer_id = ?
'''
results = cursor.execute(query, (customer_id,)).fetchall()
for result in results:
    print(result)


In [51]:
cursor.execute("INSERT INTO order_item (order_id, product_id, quantity) VALUES (?, ?, ?)", (2, 0, 1))
conn.commit()

In [52]:
query = '''
SELECT purchase_order.date, product.name, order_item.quantity
FROM purchase_order
JOIN order_item ON purchase_order.order_id = order_item.order_id
JOIN product ON order_item.product_id = product.product_id
WHERE purchase_order.customer_id = ?
'''
results = cursor.execute(query, (1,)).fetchall()
print("Final results for customer_id = 1:")
for result in results:
    print(result)

Final results for customer_id = 1:
('2020-10-20', 'bicycle', 1)


In [53]:
# Query 2: ???
# Your code here . . .
product_name = 'gloves'
query_2 = '''
SELECT customer.name, customer.email, product.name, order_item.quantity, purchase_order.date
FROM customer
JOIN purchase_order ON customer.customer_id = purchase_order.customer_id
JOIN order_item ON purchase_order.order_id = order_item.order_id
JOIN product ON order_item.product_id = product.product_id
WHERE product.name = ?
'''
results_2 = cursor.execute(query_2, (product_name,)).fetchall()
print(f"Results for customers who purchased '{product_name}':")
for result in results_2:
    print(result)

Results for customers who purchased 'gloves':
('Wendy Lee', 'wlee@bike.com', 'gloves', 2, '2020-10-20')


In [54]:
# Additional tasks
# Appending a new row to one of the tables
cursor.execute("INSERT INTO product (product_id, name, price) VALUES (?, ?, ?)", (4, 'chair', 19.99))
conn.commit()



In [55]:
rows = cursor.execute("SELECT * FROM product").fetchall()
print('data from product table')
for row in rows:
  print(row)

data from product table
(0, 'bicycle', 400.0)
(1, 'helmet', 45.0)
(2, 'gloves', 23.0)
(3, 'chain', 48.0)
(4, 'chair', 19.99)


In [56]:
# Deleting row(s) with "XXX"
cursor.execute("DELETE FROM product WHERE name = ?", ('chair',))
conn.commit()

In [57]:
rows = cursor.execute("SELECT * FROM product").fetchall()
print('data from product table')
for row in rows:
  print(row)

data from product table
(0, 'bicycle', 400.0)
(1, 'helmet', 45.0)
(2, 'gloves', 23.0)
(3, 'chain', 48.0)
